<a href="https://colab.research.google.com/github/rage-against-the-machine-learning/cs7643-DL-Group-Project/blob/plot-utilities/Homegrown_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://towardsdatascience.com/residual-network-implementing-resnet-a7da63c7b278

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
numpy_data_path = "/content/drive/MyDrive/numpy_images"

Training Data

In [ ]:
!wget https://cs7643-raml.s3.us-west-2.amazonaws.com/train_subset_np_imgs.zip
!wget https://cs7643-raml.s3.us-west-2.amazonaws.com/instances_train_subset.json

--2021-07-29 15:33:54--  https://cs7643-raml.s3.us-west-2.amazonaws.com/train_subset_np_imgs.zip
Resolving cs7643-raml.s3.us-west-2.amazonaws.com (cs7643-raml.s3.us-west-2.amazonaws.com)... 52.218.152.161
Connecting to cs7643-raml.s3.us-west-2.amazonaws.com (cs7643-raml.s3.us-west-2.amazonaws.com)|52.218.152.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5972049681 (5.6G) [application/zip]
Saving to: ‘train_subset_np_imgs.zip’

train_subset_np_img 100%[===================>]   5.56G  22.0MB/s    in 4m 40s  

2021-07-29 15:38:35 (20.3 MB/s) - ‘train_subset_np_imgs.zip’ saved [5972049681/5972049681]

--2021-07-29 15:38:35--  https://cs7643-raml.s3.us-west-2.amazonaws.com/instances_train_subset.json
Resolving cs7643-raml.s3.us-west-2.amazonaws.com (cs7643-raml.s3.us-west-2.amazonaws.com)... 52.218.176.81
Connecting to cs7643-raml.s3.us-west-2.amazonaws.com (cs7643-raml.s3.us-west-2.amazonaws.com)|52.218.176.81|:443... connected.
HTTP request sent, awaiting re

In [ ]:

! mkdir -p {numpy_data_path}/data
! unzip -q train_subset_np_imgs.zip -d {numpy_data_path}/data
! mkdir -p {numpy_data_path}/annotations
! mv instances_train_subset.json  {numpy_data_path}/annotations

Dataset and DataLoader

In [ ]:
import os
import sys
import multiprocessing as mp
import random

In [ ]:
sys.path.append("/content/drive/MyDrive/dataset")
#import coco_api_helper
#import coco_data_prep
import config_dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import torch 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
import os
import json
with open (os.path.join(numpy_data_path, "annotations","categories.json"), 'r') as j:
    desired_categories = json.load(j)
train_annot = COCO("/content/drive/MyDrive/numpy_images/annotations/instances_train2014.json")
category_ids = [cat["id"] for cat in desired_categories]
img_ids = [train_annot.getImgIds(catIds=[id]) for id in category_ids]


# flatten to a list of imgids
desired_img_ids = list(set([ii for img_id in img_ids for ii in img_id]))

loading annotations into memory...
Done (t=16.84s)
creating index...


KeyboardInterrupt: ignored

In [ ]:
os.listdir("/content/drive/MyDrive/numpy_images/data/train_subset")

In [ ]:
import os
subset_list = os.listdir("/content/drive/MyDrive/numpy_images/data/train_subset")

In [ ]:
test = train_annot.loadImgs(desired_img_ids)
subset_list = os.listdir("/content/drive/MyDrive/numpy_images/data/train_subset")
test_dict = dict(zip(desired_img_ids, [item["file_name"].replace(".jpg",".np") for item in test]))
final = {k: v for (k, v) in test_dict.items() if v in subset_list}
desired_img_ids = list(final.keys())

In [ ]:
def get_label_map(label_file):
    label_map = {}
    labels = open(label_file, 'r')
    for line in labels:
        ids = line.split(',')
        label_map[int(ids[1])] = int(ids[0])
    return label_map

In [ ]:
class COCOAnnotationTransform(object):
    """Transforms a COCO annotation into a Tensor of bbox coords and label index
    Initilized with a dictionary lookup of classnames to indexes
    """
    def __init__(self):
        self.label_map = get_label_map('/content/drive/MyDrive/dataset/coco_labels.txt')

    def __call__(self, target, width = 224, height = 224):
        """
        Args:
            target (dict): COCO target json annotation as a python dict
            height (int): height
            width (int): width
        Returns:
            a list containing lists of bounding boxes  [bbox coords, class idx]
        """
        scale = np.array([width, height, width, height])
        res = []
        for obj in target:

            if 'bbox' in obj:
                bbox = obj['bbox']
                bbox[2] += bbox[0]
                bbox[3] += bbox[1]
                label_idx = self.label_map[obj['category_id']] # -1
                final_box = list(np.array(bbox)/scale)
                final_box.append(label_idx)
                res += [final_box]  # [xmin, ymin, xmax, ymax, label_idx]
            else:
                print("no bbox problem!")

        return res[0]  # [[xmin, ymin, xmax, ymax, label_idx], ... ]

In [ ]:
class COCODataset(Dataset):
    def __init__(self,
                 np_img_data_dir,
                 annot_filepath,
                 target_transform = COCOAnnotationTransform(),
                 sample_ratio: float = None,
                 device="cpu"):
        """
        :np_img_data_dir: local directory where you have unzipped np files saved 
            np files are the trasnformed images from s3 that were normalized/ resized/ padded
        :annot_filepath: filepath of the original coco dataset corresponding to the datasplit of your choosing
        :sample_ratio: specified float between 0 and 1 for the % of images from the data split you want to use
        :device: cpu or gpu
        """
        self.device = device
        self.np_img_data_dir = np_img_data_dir

        self.sample_ratio = sample_ratio
        self.coco = COCO(annot_filepath)
        self.target_transform = target_transform

        # All possible image ids
        all_train_img_ids = list(self.coco.imgs.keys())
        # Filter down to the image ids applicable to our supercategories
        
        self.ids = [ii for ii in tqdm(all_train_img_ids) if ii in desired_img_ids]

        if self.sample_ratio is None:
            pass
        else:
            self.ids = list(np.random.choice(self.ids, int(self.sample_ratio * len(self.ids)), replace=False))

    def __getitem__(self, index):
        coco = self.coco

        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=[img_id])
        labels = coco.loadAnns(ann_ids)
        cat_list = [item["category_id"] for item in labels]
        target = max(set(cat_list), key=cat_list.count)
        target = torch.Tensor([target])

        path = coco.loadImgs([img_id])[0]['file_name']
        np_path = path.split('.')[0] + '.np'
        img = np.load(os.path.join(self.np_img_data_dir, np_path))
        
        # Convert the image to tensor so it's compatible w/ pytorch data loader
        img = torch.Tensor(img.transpose(2,0,1)).to(device=self.device).float()

        # Target transformation needs to happen to extract bounding boxes from annots
        #if self.target_transform is not None:
        #    target = torch.Tensor(self.target_transform(target))


        return img, target, labels

    def __len__(self):
        return len(self.ids)
    
def get_dataloader(dataset_obj,
                   batch_size: int=100,
                   device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                   loader_params:dict = {"shuffle" : False,
                                         "num_workers": mp.cpu_count(),
                                         "collate_fn": lambda x: x}):
    """Returns data loader for custom dataset.
    Args:
        :dataset_obj: dataset object returned from COCODataset class
        :batch_size: specified batch size, if you have memory errors while running, make this smaller
        :device: default is CUDA if there is GPU otherwise CPU
        :loader_params: default is found in config_dataset.py's dataloader_params dict
            can specify your own data loader params, but collate_fn must be `lambda x: x`
            specify:
                shuffle: True / False
                num_workers: number of parallel processes to run
                collate_fn: lambda x: x
    Returns:
        data_loader: data loader for COCODataset.
    """
    # data loader for custom dataset
    # this will return (imgs, targets) for each iteration
    loader_params.update({'batch_size': batch_size})
    
    data_loader = DataLoader(
        dataset=dataset_obj, 
        **loader_params,
    )
    return data_loader

In [ ]:
np_data_dir = "/content/drive/MyDrive/numpy_images/data/train_subset"
np_annot_dir = "/content/drive/MyDrive/numpy_images/annotations/instances_train2014.json"

In [ ]:
train_dataset = COCODataset(np_data_dir, np_annot_dir, sample_ratio=0.25)

loading annotations into memory...
Done (t=15.62s)
creating index...


  0%|          | 23/82783 [00:00<06:05, 226.23it/s]

index created!


100%|██████████| 82783/82783 [02:40<00:00, 515.78it/s]


In [ ]:
from torch.utils.data import DataLoader
from torch.multiprocessing import Pool, Process, set_start_method
import torch.multiprocessing
cpu_count = torch.multiprocessing.cpu_count()
print('cpus', cpu_count)

try: 
  set_start_method("spawn")
except RuntimeError:
  pass
batch_size = 64

train_dl = torch.utils.data.DataLoader(dataset=train_dataset, 
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=cpu_count,
                                          collate_fn=lambda x: x)

cpus 4


In [ ]:
image_batches = []
label_list = []
annot_list = []

for batch in tqdm(train_dl):
  image_batch, label_batch, annot_batch = [x[0] for x in batch], [x[1] for x in batch], [x[2] for x in batch]
  label_list.extend(label_batch)



100%|██████████| 7/7 [00:03<00:00,  2.23it/s]


ResNet

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/modules")
import resnet_modules as rn

In [ ]:
def resnet18(in_channels, n_classes, block=rn.ResNetBasicBlock, *args, **kwargs):
  return rn.ResNet(in_channels, n_classes, block=block, deepths=[2,2,2,2], *args, **kwargs)

In [ ]:
from torchsummary import summary
model = resnet18(in_channels=3,n_classes=99)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
        Conv2dAuto-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
        Conv2dAuto-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
 ResNetBasicBlock-10           [-1, 64, 56, 56]               0
       Conv2dAuto-11           [-1, 64, 56, 56]          36,864
      BatchNorm2d-12           [-1, 64, 56, 56]             128
             ReLU-13           [-1, 64, 56, 56]               0
       Conv2dAuto-14           [-1, 64,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Set Device

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(torch.cuda.current_device())
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))
    
device

device(type='cpu')

In [ ]:
import torch.nn as nn

Initialize weights

In [ ]:
def initialize_custom_block(m):
  if isinstance(m, rn.Conv2dAuto):
    torch.nn.init.xavier_uniform(m.weight)

In [ ]:
def initialize_network(m):
  if isinstance(m, nn.Conv2d):
    torch.nn.init.xavier_uniform(m.weight)
  elif isinstance(m, nn.BatchNorm2d):
    torch.nn.init.constant_(m.weight.data, 1)
    torch.nn.init.constant_(m.bias.data, 0)
  elif isinstance(m, rn.Conv2dAuto):
    torch.nn.init.xavier_uniform(m.weight)
  elif isinstance(m, rn.CustomResNetBlock):
    m.apply(initialize_custom_block)
  elif isinstance(m, rn.ResNetLayer):
    m.apply(initialize_custom_block)



In [ ]:
model.apply(initialize_network)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBasicBlock(
            (blocks): Sequential(
              (0): Sequential(
                (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU()
              (2): Sequential(
                (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [ ]:
for param in model.parameters():
  print(param)

Parameter containing:
tensor([[[[ 3.6744e-02, -2.8817e-02, -8.9449e-03,  ...,  1.4965e-03,
            1.4402e-02,  7.0007e-03],
          [ 1.5664e-02, -1.5393e-02,  1.0652e-02,  ..., -1.0610e-02,
            2.3777e-02,  5.1124e-03],
          [ 4.4250e-03,  1.8421e-02, -1.8576e-02,  ...,  2.3980e-02,
           -1.6193e-02,  1.6713e-02],
          ...,
          [ 3.3926e-03, -1.4656e-02,  1.7499e-02,  ..., -3.4492e-03,
            4.0343e-02,  1.5561e-02],
          [ 1.7671e-02, -1.6672e-03,  2.0160e-02,  ...,  3.3077e-02,
            9.4231e-03, -1.5566e-02],
          [ 1.3024e-02,  1.2205e-02,  1.1552e-02,  ...,  1.9591e-02,
           -1.9248e-03,  2.6710e-02]],

         [[ 3.5788e-02,  7.4135e-03, -2.5529e-03,  ...,  2.1407e-02,
            2.0256e-02,  6.6988e-03],
          [ 1.7754e-02, -1.7363e-02, -5.2848e-03,  ..., -6.6432e-04,
            1.5352e-02, -9.3723e-03],
          [ 3.1912e-02,  2.7844e-02,  2.3081e-02,  ...,  2.9375e-02,
            2.7260e-03,  2.4742e-02]

In [ ]:
import torch.optim as optim
import torch.nn as nn

optim_params= dict(
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-05,
    weight_decay=0.01,
    amsgrad=False,
)
optimizer = torch.optim.Adam(model.parameters(), **optim_params)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
model.to(device)
softmax = nn.Softmax()
img_id_list = []
for epoch in range(1):
  counter = 0
  running_loss = 0
  mini_batch_counter = 0
  for batch in tqdm(train_dl):
    
    # extract images and labels
    image_batch, label_batch, annot_batch = [x[0] for x in batch], [x[1] for x in batch], [x[2] for x in batch]
    img_ids_to_append = [int(item[0]) for item in label_batch]
    img_id_list.append(img_ids_to_append)
    targets = torch.LongTensor([int(item) for item in label_batch]).to(device)
    image_batch = torch.stack(image_batch).to(device)
    

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(image_batch)
    
    outputs = softmax(outputs).to(device)
    loss = criterion(outputs, targets)
    print(loss)
    loss.backward()
    optimizer.step()

    # print stats
    running_loss += loss.item()
    if counter % 10 == 9:
      print('[Epoch: %d, batch: %5d] average loss: %.3f' %
                  (epoch + 1, counter + 1, running_loss /  (counter + 1)))
      running_loss = 0.0
    counter += 1
    torch.save(model.state_dict(), "/content/drive/MyDrive/model/mini-resnet.pth")





  0%|          | 0/169 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(4.3199, grad_fn=<NllLossBackward>)



  1%|          | 1/169 [00:10<29:29, 10.53s/it]

tensor(4.1622, grad_fn=<NllLossBackward>)



  1%|          | 2/169 [00:17<26:05,  9.37s/it]

...


 99%|█████████▉| 167/169 [17:48<00:12,  6.26s/it]

tensor(4.2057, grad_fn=<NllLossBackward>)



 99%|█████████▉| 168/169 [17:54<00:06,  6.25s/it]

tensor(4.2106, grad_fn=<NllLossBackward>)



100%|██████████| 169/169 [18:00<00:00,  6.39s/it]


In [ ]:
len(train_dataset.ids)

10816

Slice model

In [ ]:
from torch import nn
from torchsummary import summary

# Dropping output layer (the ResNet classifier)

def slice_model(original_model, from_layer=None, to_layer=None):
    return nn.Sequential(*list(original_model.children())[from_layer:to_layer])
model_conv_features = slice_model(model, to_layer=-1).to(device)
summary(model_conv_features, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
        Conv2dAuto-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
        Conv2dAuto-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
 ResNetBasicBlock-10           [-1, 64, 56, 56]               0
       Conv2dAuto-11           [-1, 64, 56, 56]          36,864
      BatchNorm2d-12           [-1, 64, 56, 56]             128
             ReLU-13           [-1, 64, 56, 56]               0
       Conv2dAuto-14           [-1, 64,

Validation dataset and dataloader

In [ ]:
import os
import json
with open (os.path.join(numpy_data_path, "annotations","categories.json"), 'r') as j:
    desired_categories = json.load(j)
#with open (os.path.join(numpy_data_path, "annotations", "imgs_by_supercategory.json"), 'r') as f:
#    imgid_by_supercat = json.load(f)
valid_annot = COCO("/content/drive/MyDrive/numpy_images/annotations/instances_val2014.json")
category_ids = [cat["id"] for cat in desired_categories]
img_ids = [valid_annot.getImgIds(catIds=[id]) for id in category_ids]


# flatten to a list of imgids
desired_img_ids = list(set([ii for img_id in img_ids for ii in img_id]))

loading annotations into memory...
Done (t=8.19s)
creating index...
index created!


In [ ]:
test = valid_annot.loadImgs(desired_img_ids)
subset_list = os.listdir("/content/drive/MyDrive/numpy_images/data/valid_subset")
test_dict = dict(zip(desired_img_ids, [item["file_name"].replace(".jpg",".np") for item in test]))
final = {k: v for (k, v) in test_dict.items() if v in subset_list}
desired_img_ids = list(final.keys())

In [ ]:
np_data_dir_val = "/content/drive/MyDrive/numpy_images/data/valid_subset"
np_annot_dir_val = "/content/drive/MyDrive/numpy_images/annotations/instances_val2014.json"

In [ ]:
val_coco = COCO(np_annot_dir_val)
val_dataset = COCODataset(np_data_dir_val, np_annot_dir_val)

loading annotations into memory...
Done (t=9.71s)
creating index...
index created!
loading annotations into memory...
Done (t=5.54s)
creating index...


  0%|          | 117/40504 [00:00<00:34, 1166.52it/s]

index created!


100%|██████████| 40504/40504 [00:30<00:00, 1326.47it/s]


In [ ]:
len(val_dataset.ids)

20874

In [ ]:
from torch.utils.data import DataLoader
from torch.multiprocessing import Pool, Process, set_start_method
import torch.multiprocessing
cpu_count = torch.multiprocessing.cpu_count()
print('cpus', cpu_count)

try: 
  set_start_method("spawn")
except RuntimeError:
  pass
batch_size = 64

val_dl = torch.utils.data.DataLoader(dataset=val_dataset, 
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=cpu_count,
                                          collate_fn=lambda x: x)

cpus 4


Load Model (If required)

In [ ]:
model =  resnet18(in_channels=3,n_classes=99)
model.load_state_dict(torch.load("/content/drive/MyDrive/model/mini-resnet.pth"))



<All keys matched successfully>

Evaluate Model

In [ ]:
model_conv_features.eval()

Sequential(
  (0): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBasicBlock(
            (blocks): Sequential(
              (0): Sequential(
                (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU()
              (2): Sequential(
                (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [ ]:
features_list = []
labels_list = []
annot_list = []
image_batches = []

# Use GPUs to speed up the inference, this should take around 10 minutes

model_conv_features.to(device)
for batch in tqdm(val_dl):
    image_batch, label_batch, annot_batch = [x[0] for x in batch], [x[1] for x in batch], [x[2] for x in batch]
    image_batch = torch.stack(image_batch).to(device) 

    with torch.no_grad():
        features_batch = model_conv_features(image_batch).flatten(start_dim=1)
    features_list.append(features_batch)
    labels_list.extend(label_batch)
    annot_list.extend(annot_batch)
    image_batches.extend(image_batch)

100%|██████████| 327/327 [43:54<00:00,  8.06s/it]


In [ ]:
features = torch.cat(features_list)

In [ ]:
torch.equal(features[0], features[1])

False

In [ ]:
features.size()

torch.Size([20874, 25088])

Save embeddings

In [ ]:
image_map = [item[0]["image_id"] for item in annot_list]
embed_pos_to_image_id = dict(zip(range(len(image_map)), image_map))

In [ ]:
torch.load("/content/mini-resnet18_embeddings-v2.pt")

tensor([[0.6256, 0.5984, 0.5980,  ..., 0.1143, 0.1134, 0.1183],
        [0.6259, 0.5989, 0.5987,  ..., 0.1157, 0.1151, 0.1201],
        [0.6367, 0.7119, 0.7162,  ..., 0.0233, 0.1248, 0.0822],
        ...,
        [0.6255, 0.5981, 0.5977,  ..., 0.0514, 0.0568, 0.1260],
        [0.6260, 0.5989, 0.5987,  ..., 0.1107, 0.1105, 0.1157],
        [0.6262, 0.5924, 0.5920,  ..., 0.2199, 0.1700, 0.0825]])

In [ ]:
model_name = "mini-resnet18"
torch.save(features,"{}_embeddings-v2.pt".format(model_name))

In [ ]:
import json

with open('mini-resnet18_embeddings-v2.json', 'w') as fp:
    json.dump(embed_pos_to_image_id, fp)